In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q git+https://github.com/openai/whisper.git
!apt update && apt install -y ffmpeg
!pip install nbstripout
!pip install OpenCC

In [ ]:
from opencc import OpenCC
import whisper
import json

In [ ]:
folder_path = '/content/drive/My Drive/Ephesians_Book/'
audio_file_prefix = 'Eph'
text_file_prefix = audio_file_prefix
cc = OpenCC('t2s')
# Load the model (you can use "small", "medium", or "large" — large = best accuracy)
model = whisper.load_model("medium")  # medium is good for Chinese & faster than large

In [ ]:
def file_name_builder(folder_path, file_prefix, file_extension, index):
  return f"{folder_path}{file_prefix}_{index:02d}.{file_extension}"

In [ ]:
def convert_audio_files_to_text_json(start_index, end_index):
  for i in range(start_index, end_index):
    audio_file_name = file_name_builder(f"{folder_path}audio/",
                                        audio_file_prefix, "mp4", i)
    print(f"processing file {audio_file_name}")

    result = model.transcribe(audio_file_name, language="zh")
    # convert any traditional Chinese to simplified Chinese
    segment_object_list = transcription_result_to_objects(result)

    json_file_name = file_name_builder(f"{folder_path}text_output/",
                                      text_file_prefix, "json", i)
    with open(json_file_name, "w", encoding="utf-8") as f:
      json.dump(segment_object_list, f, ensure_ascii=False, indent=2)

In [ ]:
def transcription_result_to_objects(result):
  segment_object_list = []
  for segment in result["segments"]:
    segment_object = {
        "content": cc.convert(segment["text"]),
        "startTime": segment["start"],
        "endTime": segment["end"],
        "titleIndex": None
    }
    segment_object_list.append(segment_object)
  return segment_object_list

In [ ]:
convert_audio_files_to_text_json(59, 61)